In [ ]:
from bcpp_subject.models import HypertensionCardiovascular, SubjectLocator, SubjectConsent
from member.models import HouseholdMember

map_area = 'mmankgodi'
hypertension_cv = HypertensionCardiovascular.objects.filter(
    subject_visit__household_member__household_structure__household__plot__map_area=map_area)
subject_identifiers = []
for hcv in hypertension_cv:
    subject_identifiers.append(hcv.subject_visit.subject_identifier)

subject_identifiers = list(set(subject_identifiers))

locators = SubjectLocator.objects.filter(subject_identifier__in=subject_identifiers, may_follow_up='Yes')

requested_data = []
for locator in locators:
    firstname = ''
    lastname = ''
    consent = SubjectConsent.objects.filter(subject_identifier=locator.subject_identifier).last()
    if consent:
        firstname = consent.first_name
        lastname = consent.last_name
    else:
        member = HouseholdMember.objects.filter(subject_identifier=locator.subject_identifier).last()
        firstname = member.first_name
        lastname = member.last_name
    requested_data.append([locator.subject_identifier, firstname, lastname, locator.subject_cell])

import pandas as pd
df = pd.DataFrame(requested_data)
df.to_csv('mmankgodi_mosepele_data.csv', encoding='utf-8')